In [33]:
"""
Web3.py Lens Network Demo
This Jupyter Notebook demonstrates how to:
- Connect to the Lens Network Testnet
- Fetch the latest block number
- Check an account balance
- Send a transaction
- Perform contract call
"""

import os
from eth_account import Account
from eth_account.signers.local import LocalAccount
from web3 import Web3

## Network Information

In [ ]:
RPC_URL = "https://rpc.testnet.lens.dev"
w3 = Web3(Web3.HTTPProvider(RPC_URL))

print("Connected to Lens Network:", w3.is_connected())

latest_block = w3.eth.block_number
print("Latest Block Number:", latest_block)

## Wallet Balance

In [ ]:
my_address = ""  # Replace with your address

balance_wei = w3.eth.get_balance(my_address)
balance_grass = w3.from_wei(balance_wei, "ether")
print(f"Balance of {my_address}: {balance_grass} GRASS")


## Send Transaction

In [ ]:
# Sending a transaction (requires private key)


private_key = ""  # Replace with sender's private key
receiver_address = ""  # Replace with recipient's address

private_key = os.environ.get("PRIVATE_KEY")
assert private_key is not None, "You must set PRIVATE_KEY environment variable"
assert private_key.startswith("0x"), "Private key must start with 0x hex prefix"

account: LocalAccount = Account.from_key(private_key)

print(f"Your hot wallet address is {account.address}")

transaction = {
    "to": receiver_address,
    "from": my_address,
    "value": w3.to_wei(0.01, "ether"),  # Sending 0.01 GRASS
    "gas": 21000,
    'maxFeePerGas': w3.eth.gas_price,
    'maxPriorityFeePerGas': 1000000000,
    "nonce": w3.eth.get_transaction_count(my_address),
    "chainId": 37111,
}

signed_tx = w3.eth.account.sign_transaction(transaction, private_key)

tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
print("Transaction sent! Hash:", w3.to_hex(tx_hash))

receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Transaction confirmed in block:", receipt.blockNumber)

### Contract Call

In [ ]:
contract_address = "0xb7462EaCd5487514b6b789CF1Fca3081020F4e21"
ping_abi = [
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": True,
                "internalType": "address",
                "name": "sender",
                "type": "address",
            },
        ],
        "name": "Pong",
        "type": "event",
    },
    {
        "inputs": [],
        "name": "ping",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function",
    },
]

contract = w3.eth.contract(address=contract_address, abi=ping_abi)

transaction = contract.functions.ping().build_transaction({
    "from": my_address,
    "gas": 100000,
    "gasPrice": w3.eth.gas_price,
    "nonce": w3.eth.get_transaction_count(my_address),
    "chainId": 37111,
})

signed_tx = w3.eth.account.sign_transaction(transaction, private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
print("Ping transaction sent! Hash:", w3.to_hex(tx_hash))

receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("Transaction confirmed in block:", receipt.blockNumber)
